In [117]:
import requests
from collections import namedtuple
from hashlib import md5
import pathlib
import zipfile
from struct import *
from pytdx.reader.history_financial_reader import HistoryFinancialReader



    
def _files():
    files = requests.get('http://down.tdx.com.cn:8001/fin/gpcw.txt').text.split()
#     files = [FinFile(*f.split(',')) for f in files]
    return [f.split(',') for f in files]

current_files = _files()

def get_file(year,quarter ):
    _format = 'gpcw{year}{md}.zip'
    if quarter == 1:
        file_name = _format.format(year=year, md='0331')
    if quarter == 2:
        file_name = _format.format(year=year, md='0631')
    if quarter == 3:
        file_name = _format.format(year=year, md='0930')
    if quarter == 4:
        file_name = _format.format(year=year, md='1231')
    
    return [f for f in current_files if f[0] == file_name][0]
    

class FinFile:
    file_path = '/data/stock/tdx/finzips/'

    def __init__(self, file_name, md5, size):
        self.file_name = file_name
        self.md5 = md5
        self.size = size
        
    @property
    def down_url(self):
        return 'http://down.tdx.com.cn:8001/fin/{file_name}'.format(file_name=self.file_name)
    
    @property
    def cache_file_name(self):
        return self.file_path + self.file_name
    
    @property
    def cache_file_md5(self):
        if not pathlib.Path(self.cache_file_name).exists():
            return 'x'
        m = md5()
        with open(self.cache_file_name, 'rb') as f:
            m.update(f.read())
        return m.hexdigest()
    
    def _down_load(self):
        print(f'down {self.down_url} ==> {self.cache_file_name}')
        r = requests.get(self.down_url, stream=True)
        with open(self.cache_file_name, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: 
                    f.write(chunk)
            
        zip_file = zipfile.ZipFile(self.cache_file_name)
        zip_file.extractall(path=self.file_path)
            

    def __enter__(self):
        for f in current_files:
            if f[0] == self.file_name:
                if f[1] != self.cache_file_md5:
                    self._down_load()
        
        return self
    
    def __exit__(self, exc_type, exc, trace_info):
        pass
    
    def to_df(self):
        datafile = self.cache_file_name.replace('.zip','.dat')
        df = HistoryFinancialReader().get_df(datafile)
        
 

def get_fin(year, quarter):
    f = get_file(year, quarter)
    with FinFile(*f) as f1:
        return f1.to_df()

In [116]:
df = get_fin('2018', 1)

In [113]:
df

,report_date,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col273,col274,col275,col276,col277,col278,col279,col280,col281,col282
code,,,,,,,,,,,,,,,,,,,,,
000001,20180331,0.3300,0.3300,4.69,11.850000,3.290000e+00,3.242,2.410,6.699981e+11,7.684500e+10,...,-4.039810e+34,2.000000e+00,2.775952e+08,2.357000e+10,-4.039810e+34,-4.039810e+34,3.000000e+00,9.828775e+08,2.790,0.3818
000002,20180331,0.0810,0.0748,7.31,12.336600,7.500000e-01,0.657,-2.514,9.478232e+10,-4.039810e+34,...,-4.039810e+34,1.000000e+00,3.242811e+09,2.825128e+10,-4.039810e+34,-4.039810e+34,-4.039810e+34,-4.039810e+34,0.660,0.0748
000004,20180331,0.0245,0.0035,0.43,1.568600,1.000000e-02,1.564,0.106,1.354697e+08,-4.039810e+34,...,-4.039810e+34,1.000000e+00,2.387685e+07,1.180520e+07,-4.039810e+34,-4.039810e+34,1.000000e+00,8.397600e+06,1.580,0.0035
000005,20180331,0.0153,0.0155,-0.41,1.264600,6.500000e-01,1.206,0.003,8.953328e+07,-4.039810e+34,...,-4.039810e+34,6.000000e+00,2.317194e+08,4.159604e+07,-4.039810e+34,-4.039810e+34,1.000000e+00,1.300235e+07,1.210,0.0155
000006,20180331,0.2120,0.2117,1.97,4.336300,3.600000e-01,4.889,-0.111,9.616100e+08,0.000000e+00,...,-4.039810e+34,1.000000e+00,1.629668e+08,9.879852e+08,-4.039810e+34,-4.039810e+34,1.000000e+00,4.736820e+07,5.010,0.2118
000007,20180331,-0.0200,-0.0014,-0.38,1.094500,4.500000e-01,-2.040,-0.013,4.411901e+06,4.913852e+07,...,-4.039810e+34,5.000000e+00,9.291038e+07,8.441896e+06,1.000000e+00,1.380813e+07,-4.039810e+34,-4.039810e+34,-2.020,-0.0014
000008,20180331,0.0037,0.0032,0.54,2.527500,1.030000e+00,0.148,-0.147,7.798847e+08,3.953912e+05,...,-4.039810e+34,6.000000e+00,7.888042e+08,8.749635e+08,-4.039810e+34,-4.039810e+34,-4.039810e+34,-4.039810e+34,0.150,0.0033
000009,20180331,0.0164,0.0478,0.86,2.331800,3.700000e-01,0.703,0.047,3.355398e+09,9.726692e+08,...,-4.039810e+34,2.000000e+00,3.758013e+08,1.349654e+08,-4.039810e+34,-4.039810e+34,1.000000e+00,6.414788e+06,0.350,0.0478
000010,20180331,-0.0568,-0.0568,-1.49,1.351200,1.820000e+00,-4.204,-0.102,3.570341e+08,-4.039810e+34,...,-4.039810e+34,4.000000e+00,7.694842e+07,-1.082483e+09,4.000000e+00,2.241880e+07,1.000000e+00,3.472621e+06,-4.120,-0.0568


Object `zip_file` not found.


In [36]:
z = zipfile.ZipFile('/data/stock/tdx/finzips/gpcw19960630.zip')

In [39]:
?z.extractall